In [156]:
import os
api_key = os.environ.get('FLIGHT_API_KEY', '')
from typing import List

import requests
import pandas as pd

from pprint import pprint
from datetime import datetime

In [157]:
api_key

'9a7cdb-87fe76'

### PLAN:
##### 1.	“ SHOW ME ALL THE FLIGHT SCHEDULE THAT DEPARTS FROM DENVER FOR TODAY ON A MAP (D3.js)”
##### 2.	“AND COMPARE THE SCHEDULE WITH ITS ACTUAL DATA TO DISPLAY PROGRESS ON THE SAME MAP”

*	Display the scheduled flight time <code><strong>(/schedule route)</strong></code> and its actual data <strong><code>(/projection route)</code></strong> onto the same map
*	Estimated arrival time using Flight distance API calculation a distance (or time by speed) between flight’s current location and its destination
*	Other possibilities: time skewness (estimated flying time - actual flying time) by airlines as a performance metric

### TECHNOLOGIES:
* Local development: FLASK + MONGODB
* Web deployment: Google Cloud?

#### 1. Design for FLIGHT TRACKER API
Required fields:
* departure - iataCode
* arrival - iataCode
* flight - iataNumber
* aircraft - regNumber, iataCode
* geography - altitude, direction, latitude, longitude
* speed - horizontal, vertical, isGround
* status
* system - squawk, updated

In [195]:
base_url  = 'http://aviation-edge.com/v2/public/flights'
params = { 'key': api_key,
           'depIata': 'DEN',
#           'depIcao': '',
#           'arrIata': '',
#           'arrIcao': '',
#           'aircraftIcao': '',
#           'regNum': '',
#           'aircraftIcao24': '',
#           'airlineIata': '',
#           'airlineIcao': '',
#           'flightIata': '',
#           'flightIcao': '',
#           'flightNum': '',
#           'status': '',
#           'limit': 100
        }

resp_flight = requests.get(base_url, params=params).json()

In [196]:
len(resp_flight)

367

In [197]:
pprint(resp_flight[3])

{'aircraft': {'iataCode': 'B738',
              'icao24': '',
              'icaoCode': 'B738',
              'regNumber': 'N76503'},
 'airline': {'iataCode': 'UA', 'icaoCode': 'UAL'},
 'arrival': {'iataCode': 'LGA', 'icaoCode': 'KLGA'},
 'departure': {'iataCode': 'DEN', 'icaoCode': 'KDEN'},
 'flight': {'iataNumber': 'UA406', 'icaoNumber': 'UAL406', 'number': '406'},
 'geography': {'altitude': 11277.6,
               'direction': 88.34,
               'latitude': 42.0506,
               'longitude': -88.4931},
 'speed': {'horizontal': 1024.6, 'isGround': 0, 'vertical': 0},
 'status': 'en-route',
 'system': {'squawk': '6261', 'updated': '1569454200'}}


#### 2. Design for AIRPORT DATABASE API
Required fields:
* codeIataAirport, codeIataCity, codeIso2Country
* nameAirport, nameCountry
* latitudeAirport, longitudeAirport
* timezone(?)

In [230]:
base_url = 'https://aviation-edge.com/v2/public/airportDatabase'
params = {'key': api_key,
          'codeIso2Country': 'US'
         }

us_airports = requests.get(base_url, params=params).json()

In [17]:
len(us_airports)

2336

In [ ]:
# Display all airport results containing user-defined strings
pprint([airport for airport in us_airports if ('denver' in airport.get('timezone').lower()) and ('international' in airport.get('nameAirport').lower())])


In [34]:
# Sample data

# DENVER INTERNATIONAL AIRPORT (DIA)

#   'GMT': '-7',
#   'airportId': '1746',
#   'codeIataAirport': 'DEN',
#   'codeIataCity': 'DEN',
#   'codeIcaoAirport': 'KDEN',
#   'codeIso2Country': 'US',
#   'geonameId': '5419401',
#   'latitudeAirport': '39.84939',
#   'longitudeAirport': '-104.672844',
#   'nameAirport': 'Denver International',
#   'nameCountry': 'United States',
#   'phone': '303-342-2000',
#   'timezone': 'America/Denver'},

#### 3. Design for TIMETABLE API
Required fields ('departure'):
* departure - iataCode, scheduledTime, gate, terminal, delay (if delayed)
* arrival - iataCode, scheduledTime, gate, terminal, baggage, delay (if delayed)
* airline
* flight number (iata?)
* status

In [239]:
base_url = 'http://aviation-edge.com/v2/public/timetable'
params = {'key': api_key,
          'iataCode': 'DEN',
          'type': 'departure'
         }

resp_timetable = requests.get(base_url, params=params).json()

In [240]:
len(resp_timetable)

748

In [194]:
resp_timetable[3]

{'type': 'departure',
 'status': 'active',
 'departure': {'iataCode': 'DEN',
  'icaoCode': 'KDEN',
  'terminal': None,
  'gate': 'B29',
  'baggage': None,
  'delay': None,
  'scheduledTime': '2019-09-25T22:50:00.000',
  'estimatedTime': '2019-09-25T22:50:00.000',
  'actualTime': None,
  'estimatedRunway': None,
  'actualRunway': None},
 'arrival': {'iataCode': 'BOS',
  'icaoCode': 'KBOS',
  'terminal': 'B',
  'gate': 'B27',
  'baggage': None,
  'delay': None,
  'scheduledTime': '2019-09-26T04:46:00.000',
  'estimatedTime': None,
  'actualTime': None,
  'estimatedRunway': None,
  'actualRunway': None},
 'airline': {'name': 'United Airlines', 'iataCode': 'UA', 'icaoCode': 'UAL'},
 'flight': {'number': '1606', 'iataNumber': 'UA1606', 'icaoNumber': 'UAL1606'},
 'codeshared': None}

In [191]:
pprint([schedule['flight'] for schedule in resp if schedule.get('flight').get('iataNumber') == "UA3634"])

[{'iataNumber': 'UA3634', 'icaoNumber': 'UAL3634', 'number': '3634'}]


In [251]:
pprint([schedule for schedule in resp_flight if schedule.get('speed').get('isGround') != 0])

[{'aircraft': {'iataCode': 'B752',
               'icao24': '4CC268',
               'icaoCode': 'B752',
               'regNumber': 'TFFIC'},
  'airline': {'iataCode': 'FI', 'icaoCode': 'ICE'},
  'arrival': {'iataCode': 'KEF', 'icaoCode': 'BIKF'},
  'departure': {'iataCode': 'DEN', 'icaoCode': 'KDEN'},
  'flight': {'iataNumber': 'FI670', 'icaoNumber': 'ICE670', 'number': '670'},
  'geography': {'altitude': 7658.1,
                'direction': 44,
                'latitude': 40.672,
                'longitude': -104.117},
  'speed': {'horizontal': 755.616, 'isGround': 1, 'vertical': 8.45312},
  'status': 'en-route',
  'system': {'squawk': '1445', 'updated': '1569453605'}}]


In [209]:
match_count = 0

flight_numbers_in_timetable = {flight.get('flight').get('icaoNumber') for flight in resp_timetable}
flight_numbers_in_tracker = {flight.get('flight').get('icaoNumber') for flight in resp_flight}

for flight in resp_flight:
    if flight.get('flight').get('icaoNumber') in flight_numbers_in_timetable:
        match_count += 1

In [210]:
match_count

180

In [206]:
len(flight_numbers_in_timetable)

1019

In [208]:
len(flight_numbers_in_tracker)

367

In [132]:
def get_info_timetable(airport_code: str ='DEN', airline_code: str =None):
    
    results = []
    
    base_url = 'http://aviation-edge.com/v2/public/timetable'
    params = {'key': api_key,
              'iataCode': airport_code,
              'airline_iata': airline_code,
              'type': 'departure'
             }

    if not airline_code: del params['airline_iata']
        
    schedules = requests.get(base_url, params=params).json()
    print(f'Searched data counts: {len(schedules)}')

    for schedule in schedules:

        # departure info
        departure_iatacode = schedule['departure']['iataCode']
        departure_scheduledtime = schedule['departure']['scheduledTime']
        departure_scheduledtime = datetime.strptime(departure_scheduledtime, '%Y-%m-%dT%H:%M:%S.%f').strftime('%Y-%m-%d %H:%M:%S')
        departure_gate = schedule['departure']['gate']
        departure_terminal = schedule['departure']['terminal']
        departure_delay = schedule['departure']['delay']

        # arrival info
        arrival_iatacode = schedule['arrival']['iataCode']
        arrival_scheduledtime = schedule['arrival']['scheduledTime']
        arrival_scheduledtime = datetime.strptime(arrival_scheduledtime, '%Y-%m-%dT%H:%M:%S.%f').strftime('%Y-%m-%d %H:%M:%S')
        arrival_gate = schedule['arrival']['gate']
        arrival_terminal = schedule['arrival']['terminal']
        arrival_baggage = schedule['arrival']['baggage']
        arrival_delay = schedule['arrival']['delay']

        # airline info
        airline_iatacode = schedule['airline']['iataCode']
        airline_name = schedule['airline']['name']

        # flight info
        flight_iatanumber = schedule['flight']['iataNumber']
        flight_status = schedule['status']
        
        results.append(
             {
              'departure_iatacode': departure_iatacode,
              'departure_scheduledtime': departure_scheduledtime,
              'departure_gate': departure_gate,
              'departure_terminal': departure_terminal,
              'departure_delay': departure_delay,
              'arrival_iatacode': arrival_iatacode,
              'arrival_scheduledtime': arrival_scheduledtime,
              'arrival_gate': arrival_gate,
              'arrival_terminal': arrival_terminal,
              'arrival_baggage': arrival_baggage,
              'arrival_delay': arrival_delay,
              'airline_iatacode': airline_iatacode,
              'airline_name': airline_name,
              'flight_iatanumber': flight_iatanumber,
              'flight_status': flight_status  
            }
        )

    return results


In [133]:
results = get_info_timetable('DEN')

Searched data counts: 602


In [149]:
schedule_df = pd.DataFrame.from_records(results)

In [151]:
schedule_df.query("flight_iatanumber == 'WN150'")

,airline_iatacode,airline_name,arrival_baggage,arrival_delay,arrival_gate,arrival_iatacode,arrival_scheduledtime,arrival_terminal,departure_delay,departure_gate,departure_iatacode,departure_scheduledtime,departure_terminal,flight_iatanumber,flight_status
49,WN,Southwest Airlines,None,None,406,ONT,2019-09-24 22:35:00,4,35,C37,DEN,2019-09-24 21:25:00,None,WN150,active


#### Airport Database
Required fields ('departure'):

In [228]:
base_url = 'https://aviation-edge.com/v2/public/airportDatabase'
params = {'key': api_key}

airport_db = requests.get(base_url, params=params).json()

In [215]:
airport_db[4]

{'airportId': '5',
 'nameAirport': 'Apalachicola Regional',
 'codeIataAirport': 'AAF',
 'codeIcaoAirport': 'KAAF',
 'latitudeAirport': '29.733334',
 'longitudeAirport': '-84.98333',
 'geonameId': '4146153',
 'timezone': 'America/New_York',
 'GMT': '-5',
 'phone': '',
 'nameCountry': 'United States',
 'codeIso2Country': 'US',
 'codeIataCity': 'AAF'}

In [113]:
df_airport = pd.DataFrame.from_records(airport_db)
df_airport.head()

,GMT,airportId,codeIataAirport,codeIataCity,codeIcaoAirport,codeIso2Country,geonameId,latitudeAirport,longitudeAirport,nameAirport,nameCountry,phone,timezone
0,-10,1,AAA,AAA,NTGA,PF,6947726,-17.05,-145.41667,Anaa,French Polynesia,,Pacific/Tahiti
1,10,2,AAB,AAB,YARY,AU,7730796,-26.7,141.04167,Arrabury,Australia,,Australia/Brisbane
2,2,3,AAC,AAC,HEAR,EG,6297289,31.133333,33.75,El Arish International Airport,Egypt,,Africa/Cairo
3,1,4,AAE,AAE,DABB,DZ,2570559,36.821392,7.811857,Les Salines,,,Africa/Algiers
4,-5,5,AAF,AAF,KAAF,US,4146153,29.733334,-84.98333,Apalachicola Regional,United States,,America/New_York


In [115]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///airport.db', echo=True)

In [116]:
df_airport.to_sql('airports', con=engine)

2019-09-21 16:17:17,190 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-09-21 16:17:17,190 INFO sqlalchemy.engine.base.Engine ()
2019-09-21 16:17:17,191 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-09-21 16:17:17,192 INFO sqlalchemy.engine.base.Engine ()
2019-09-21 16:17:17,193 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("airports")
2019-09-21 16:17:17,193 INFO sqlalchemy.engine.base.Engine ()
2019-09-21 16:17:17,196 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE airports (
	"index" BIGINT, 
	"GMT" TEXT, 
	"airportId" TEXT, 
	"codeIataAirport" TEXT, 
	"codeIataCity" TEXT, 
	"codeIcaoAirport" TEXT, 
	"codeIso2Country" TEXT, 
	"geonameId" TEXT, 
	"latitudeAirport" TEXT, 
	"longitudeAirport" TEXT, 
	"nameAirport" TEXT, 
	"nameCountry" TEXT, 
	phone TEXT, 
	timezone TEXT
)


2019-09-21 16:17:17,197 INFO sqlalchemy.engine.base.Engine ()
2019-09-21 16:17:17,199 INFO sqlalchem

In [119]:
%%bash
open .

In [ ]:
engine.execute('SELECT * FROM airports').fetchall()

In [120]:
df = pd.read_sql_query('SELECT * FROM airports', engine)

2019-09-21 23:30:34,822 INFO sqlalchemy.engine.base.Engine SELECT * FROM airports
2019-09-21 23:30:34,822 INFO sqlalchemy.engine.base.Engine ()


In [ ]:
df

In [218]:
base_url = "http://aviation-edge.com/v2/public/routes"
params = {"key": api_key}

resp_route = requests.get(base_url, params=params).json()

In [219]:
len(resp_route)

300

In [ ]:
den_route = [route for route in resp_route if route.get('departureIata') == "OTP" ]
den_route